# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well! UC Irvine 

In [26]:
import findspark

findspark.init('/home/guipleite/spark-3.0.2-bin-hadoop3.2')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lr_exe').getOrCreate()

In [27]:
df = spark.read.csv("cruise_ship_info.csv",inferSchema=True,header=True)

In [28]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [29]:
df.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [30]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer


In [31]:


indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_line_idx")
df_idx = indexer.fit(df).transform(df)
df_idx.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_idx|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [32]:
assembler = VectorAssembler(
    inputCols=["Age", "Tonnage", 
               "passengers",'length', 'cabins',
                'passenger_density', 'Cruise_line_idx'],
    outputCol="features")

output = assembler.transform(df_idx)

final_data = output.select("features",'crew')
final_data.show(5)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [33]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [34]:
lr = LinearRegression(labelCol='crew')
lrModel = lr.fit(train_data)

In [35]:
test_results = lrModel.evaluate(test_data)
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -1.259683059438844|
| -1.6675104957939713|
| -0.8284943054002669|
|  -0.663985479303248|
| -0.5918948409887062|
| -1.3323210289365939|
| -0.5299002462038302|
|  -0.678389153864698|
| -0.6000419341977352|
|   0.969418618381571|
|  1.7250189665273483|
|-0.19440941479296026|
| -0.3051406074438763|
|-0.49299754202097645|
| 0.18365977946264955|
|-0.16618942603373021|
| -0.3953511231717268|
|   0.761301372839629|
| 0.10882883494021778|
|  0.8022472563108547|
+--------------------+
only showing top 20 rows



In [36]:
unlabeled_data = test_data.select('features')
predictions = lrModel.transform(unlabeled_data)

predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.259683059438844|
|[5.0,160.0,36.34,...|15.267510495793971|
|[6.0,30.276999999...| 4.378494305400267|
|[6.0,112.0,38.0,9...|11.563985479303248|
|[6.0,158.0,43.7,1...|14.191894840988706|
|[7.0,89.6,25.5,9....|11.202321028936593|
|[7.0,158.0,43.7,1...| 14.12990024620383|
|[9.0,85.0,19.68,9...| 9.368389153864698|
|[9.0,110.0,29.74,...|12.200041934197735|
|[9.0,113.0,26.74,...| 11.41058138161843|
|[10.0,46.0,7.0,6....|2.7449810334726514|
|[10.0,68.0,10.8,7...| 6.554409414792961|
|[10.0,81.76899999...| 8.725140607443876|
|[10.0,86.0,21.14,...| 9.692997542020976|
|[11.0,58.6,15.66,...|  7.41634022053735|
|[11.0,86.0,21.24,...| 9.466189426033731|
|[11.0,90.09,25.01...| 8.875351123171727|
|[11.0,91.0,20.32,...| 9.228698627160371|
|[12.0,25.0,3.88,5...|2.7611711650597823|
|[12.0,77.104,20.0...| 8.787752743689145|
+--------------------+------------

In [37]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 0.91411148769994
MSE: 0.8355998119449976


In [38]:
train_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              103|
|   mean| 7.69747572815535|
| stddev|3.718665351568239|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+

